In [248]:
###  " 기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(240715).xlsx"

In [257]:
import pandas as pd
import numpy as np
import requests
import time
import json
import matplotlib.pyplot as plt

In [258]:
df_weather = pd.read_excel('기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(240715).xlsx') 

In [259]:
df_weather.head(10)

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN
5,kor,1111055000,서울특별시,종로구,부암동,60,127,126,57,59.20,37,35,23.48,126.966444,37.589856,NaN
6,kor,1111056000,서울특별시,종로구,평창동,60,127,126,58,7.96,37,36,9.08,126.968878,37.602522,NaN
7,kor,1111057000,서울특별시,종로구,무악동,60,127,126,57,40.35,37,34,17.54,126.961208,37.571539,NaN
8,kor,1111058000,서울특별시,종로구,교남동,60,127,126,57,50.76,37,34,8.67,126.964100,37.569075,NaN
9,kor,1111060000,서울특별시,종로구,가회동,60,127,126,59,12.88,37,34,38.11,126.986911,37.577253,NaN


In [260]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3830 entries, 0 to 3829
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   구분         3830 non-null   object 
 1   행정구역코드     3830 non-null   int64  
 2   1단계        3830 non-null   object 
 3   2단계        3811 non-null   object 
 4   3단계        3559 non-null   object 
 5   격자 X       3830 non-null   int64  
 6   격자 Y       3830 non-null   int64  
 7   경도(시)      3830 non-null   int64  
 8   경도(분)      3830 non-null   int64  
 9   경도(초)      3830 non-null   float64
 10  위도(시)      3830 non-null   int64  
 11  위도(분)      3830 non-null   int64  
 12  위도(초)      3830 non-null   float64
 13  경도(초/100)  3830 non-null   float64
 14  위도(초/100)  3830 non-null   float64
 15  위치업데이트     945 non-null    float64
dtypes: float64(5), int64(7), object(4)
memory usage: 478.9+ KB


In [261]:
# df_pohang_xy_01 = df_pohang_xy.query("'2단계' == '포항시남구'")  
df_pohang = df_weather[ (df_weather.iloc[:,3] == '포항시남구') | (df_weather.iloc[:,3] == '포항시북구')  ]

In [262]:
df_pohang.head()

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
2637,kor,4711100000,경상북도,포항시남구,NaN,102,94,129,21,42.00,36,0,20.47,129.361667,36.005686,NaN
2638,kor,4711125000,경상북도,포항시남구,구룡포읍,105,94,129,32,54.71,35,58,55.47,129.548531,35.982075,NaN
2639,kor,4711125300,경상북도,포항시남구,연일읍,102,94,129,20,47.11,35,59,19.05,129.346419,35.988625,NaN
2640,kor,4711125600,경상북도,포항시남구,오천읍,103,93,129,25,1.32,35,57,36.71,129.417033,35.960197,NaN
2641,kor,4711131000,경상북도,포항시남구,대송면,102,93,129,21,44.08,35,57,55.78,129.362244,35.965494,NaN


In [263]:
# df_pohang = df_pohang.dropna(subset=['3단계'], inplace=True)
# df_pohang

In [264]:
'''
http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst
?serviceKey=인증키&numOfRows=10&pageNo=1
&base_date=20210628&base_time=0600&nx=55&ny=127

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=인증키                                                                                   &numOfRows=10&pageNo=1&base_date=20210628&base_time=0600&nx=55&ny=127
http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=929xzfpOxaYkGzWQ2koOHVPCPmzhY00lTRcU2uonKEG+mcdXzwQ7sSiH1jwAcj78fHV/XHhl0U+N76T8OSsT2A==&numOfRows=10&pageNo=1&base_date20240803&base_time=0600&dataType=JSON&nx=102&ny=94

'''

'\nhttp://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst\n?serviceKey=인증키&numOfRows=10&pageNo=1\n&base_date=20210628&base_time=0600&nx=55&ny=127\n\nhttp://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=인증키                                                                                   &numOfRows=10&pageNo=1&base_date=20210628&base_time=0600&nx=55&ny=127\nhttp://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=929xzfpOxaYkGzWQ2koOHVPCPmzhY00lTRcU2uonKEG+mcdXzwQ7sSiH1jwAcj78fHV/XHhl0U+N76T8OSsT2A==&numOfRows=10&pageNo=1&base_date20240803&base_time=0600&dataType=JSON&nx=102&ny=94\n\n'

In [265]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey='  # [초단기실황조회] 
apiKey = '929xzfpOxaYkGzWQ2koOHVPCPmzhY00lTRcU2uonKEG%2BmcdXzwQ7sSiH1jwAcj78fHV%2FXHhl0U%2BN76T8OSsT2A%3D%3D'  # Open API ; Encoding Key
# apiKey = '929xzfpOxaYkGzWQ2koOHVPCPmzhY00lTRcU2uonKEG+mcdXzwQ7sSiH1jwAcj78fHV/XHhl0U+N76T8OSsT2A=='  # Open API ; Decoding Key
base_date = '20240803'
base_time = '1500'
dataType = 'JSON'  # 출력 결과 형식(xml, json / 기본: xml)

In [266]:
url_01 = url + apiKey + '&numOfRows=10&pageNo=1' + '&base_date=' + base_date + '&base_time=' + base_time + '&dataType=' + dataType
url_01

'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=929xzfpOxaYkGzWQ2koOHVPCPmzhY00lTRcU2uonKEG%2BmcdXzwQ7sSiH1jwAcj78fHV%2FXHhl0U%2BN76T8OSsT2A%3D%3D&numOfRows=10&pageNo=1&base_date=20240803&base_time=1500&dataType=JSON'

In [267]:
def get_request_url():
    response = requests.get(full_url)
    print(response.status_code)
    # response.content # => response.content는 한글이 인코딩된 형식이므로
    #                       response.text 를 응답받기로함
    return response.text   #response.text

In [268]:

# Creating Empty DataFrame and Storing it in variable df
df0 = pd.DataFrame(columns=['BaseDate', 'ObsTime', 'nx', 'ny', 'longitude', 'latitude', 'degree', 'wind_speed', 'w_dir'])
print(df0)


Empty DataFrame
Columns: [BaseDate, ObsTime, nx, ny, longitude, latitude, degree, wind_speed, w_dir]
Index: []


In [269]:
for i in range(len(df_pohang)):
    x = str(df_pohang.iloc[i]['격자 X'])
    y = str(df_pohang.iloc[i]['격자 Y'])
    print('iter', i)
    # ['BaseDate', 'ObsTime', 'nx', 'ny', 'longitude', 'latitude', 'degree', 'wind_speed', 'w_dir']
    df0.at[i,'BaseDate'] = base_date
    df0.at[i,'ObsTime'] = base_time    
    df0.at[i,'nx'] = x
    df0.at[i,'ny'] = y  
    df0.at[i,'longitude'] = df_pohang.iloc[i]['경도(초/100)'] # 경도
    df0.at[i,'latitude'] = df_pohang.iloc[i]['위도(초/100)'] # 위도   
    full_url = url_01 + '&nx=' + x + '&ny=' + y
    # print(full_url)
    raw_str_json = get_request_url()
    if raw_str_json:
        raw_json = json.loads(raw_str_json)
    parsed_json = raw_json['response']['body']['items']['item']
    df = pd.DataFrame(parsed_json)


    df0.at[i,'degree'] = df.iloc[3]['obsrValue']  #.astype(float64) #  degree (℃)
    df0.at[i,'wind_speed'] = df.iloc[7]['obsrValue']  #.astype(float64) # wind speed (m/s)
    df0.at[i,'w_dir'] = df.iloc[6]['obsrValue']  #.astype(float64) # wind direction (deg)
 

    # result = pd.concat([df1, df4])   # , ignore_index=True, sort=False)    
    print(df0)

iter 0
200
   BaseDate ObsTime   nx  ny   longitude   latitude degree wind_speed w_dir
0  20240803    1500  102  94  129.361667  36.005686   33.9        3.1   0.4
iter 1
200
   BaseDate ObsTime   nx  ny   longitude   latitude degree wind_speed w_dir
0  20240803    1500  102  94  129.361667  36.005686   33.9        3.1   0.4
1  20240803    1500  105  94  129.548531  35.982075   31.5        2.8   2.7
iter 2
200
   BaseDate ObsTime   nx  ny   longitude   latitude degree wind_speed w_dir
0  20240803    1500  102  94  129.361667  36.005686   33.9        3.1   0.4
1  20240803    1500  105  94  129.548531  35.982075   31.5        2.8   2.7
2  20240803    1500  102  94  129.346419  35.988625   33.9        3.1   0.4
iter 3
200
   BaseDate ObsTime   nx  ny   longitude   latitude degree wind_speed w_dir
0  20240803    1500  102  94  129.361667  36.005686   33.9        3.1   0.4
1  20240803    1500  105  94  129.548531  35.982075   31.5        2.8   2.7
2  20240803    1500  102  94  129.346419  35

In [270]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BaseDate    31 non-null     object
 1   ObsTime     31 non-null     object
 2   nx          31 non-null     object
 3   ny          31 non-null     object
 4   longitude   31 non-null     object
 5   latitude    31 non-null     object
 6   degree      31 non-null     object
 7   wind_speed  31 non-null     object
 8   w_dir       31 non-null     object
dtypes: object(9)
memory usage: 3.5+ KB


In [271]:
!pip install shapely
!pip install geopandas
!pip install folium

In [272]:
import shapely
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
import folium as f
from folium.plugins import HeatMap
import branca.colormap as cm
from collections import defaultdict

In [284]:
# df0.head(50)

* epsg : 4326 '위경도좌표계'
* epsg : 5181 '대한민국 좌표계 중 하나' 대부분의 정부 제공 오픈데이터는 이 좌표계를 사용 ⇒  위경도 데이터로 바꾸려면 좌표계 변환이 필요
* epsg : 5179 '대한민국 미터단위 좌표계' 미터단위의 처리를 할 때 이 좌표계로의 변환이 필요

In [274]:
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['degree'] = DataFrame['degree'].astype(float)
    DataFrame['latitude'] = DataFrame['latitude'].astype(float)
    DataFrame['longitude'] = DataFrame['longitude'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['longitude'], row['latitude']]), axis=1)
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'}
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326
    return DataFrame

In [285]:
df_geo = geo_transform(df0)
df_geo.head(10)

C:\Users\IT\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,BaseDate,ObsTime,nx,ny,longitude,latitude,degree,wind_speed,w_dir,geometry
0,20240803,1500,102,94,129.361667,36.005686,33.9,3.1,0.4,POINT (129.36167 36.00569)
1,20240803,1500,105,94,129.548531,35.982075,31.5,2.8,2.7,POINT (129.54853 35.98208)
2,20240803,1500,102,94,129.346419,35.988625,33.9,3.1,0.4,POINT (129.34642 35.98862)
3,20240803,1500,103,93,129.417033,35.960197,33.2,1.7,-0.8,POINT (129.41703 35.9602)
4,20240803,1500,102,93,129.362244,35.965494,33.9,3.1,0.4,POINT (129.36224 35.96549)
5,20240803,1500,104,94,129.442542,35.985508,31.5,2.8,2.7,POINT (129.44254 35.98551)
6,20240803,1500,105,92,129.494919,35.892778,31.5,2.8,2.7,POINT (129.49492 35.89278)
7,20240803,1500,106,96,129.555531,36.080854,34.3,4.3,3.2,POINT (129.55553 36.08085)
8,20240803,1500,102,94,129.356963,36.014019,33.9,3.1,0.4,POINT (129.35696 36.01402)
9,20240803,1500,102,94,129.371447,36.023286,33.9,3.1,0.4,POINT (129.37145 36.02329)


In [293]:
steps = 100
min_temp = df_geo['degree'].min()
max_temp = df_geo['degree'].max()
color_map= cm.LinearColormap(
    colors=['blue', 'green', 'yellow', 'orange', 'red'], # heatmap 색상
    # vmin=min_temp, vmax=max_temp  
    vmin=0.5, vmax=0.7
).to_step(n=200)

# color_map = cm.linear.YlOrRd_09.scale(min_temp, max_temp).to_step(n=100)
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

In [297]:
# Draw a basemap
m = f.Map(location=[36, 129.3], tiles='openstreetmap', zoom_start=11)

# Add heatmap
HeatMap(
    data=df_geo[['latitude', 'longitude']], 
    radius=10,
    gradient=gradient_map
).add_to(m)
    
# Display the map
m
color_map= cm.LinearColormap(
    colors=['blue', 'green', 'yellow', 'orange', 'red'], # heatmap 색상 
    vmin=min_temp, vmax=max_temp
).to_step(n=200)
color_map.caption='포항지역 도심 열섬 현상의 분석  (℃)' # colorbar 문구 설정
m.add_child(color_map)